In [1]:
import sys
from osgeo import gdal
gdal.VersionInfo()

'3060200'

In [9]:
import logging
from pathlib import Path
import wget
import glob
import numpy as np
import rasterio
import pyproj
import xarray as xr
from osgeo import gdal
from osgeo import osr
import netCDF4
import os

In [2]:
print(sys.path)

['/Users/khant/Documents/UFZ/Git/iasdt-workflows/notebooks', '/Users/khant/opt/anaconda3/lib/python39.zip', '/Users/khant/opt/anaconda3/lib/python3.9', '/Users/khant/opt/anaconda3/lib/python3.9/lib-dynload', '', '/Users/khant/.local/lib/python3.9/site-packages', '/Users/khant/opt/anaconda3/lib/python3.9/site-packages', '/Users/khant/opt/anaconda3/lib/python3.9/site-packages/aeosa']


In [3]:
# some_file.py
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
if "../src/data/" in sys.path:
    print("Path not needed")
else:
    sys.path.insert(1, "../src/data/")


In [8]:
def download_data(path_to_download_list, output_dir):
    """Downloads the CHELSA yearly data from the C3S FTP server.

    :param path_to_download_list: The path for the txt file with CHELSA data url list.
    :param: output_dir: The path where data files are downloaded to.

    Returns:
        None
    """
    logger = logging.getLogger(__name__)
    logger.info("downloading CHELSA data...")
    # Download the CHELSA data from the C3S FTP server
    with open(path_to_download_list) as url_list:
        for line in url_list:
            response = wget.download(line, out=output_dir)
            logger.info("downlaoded CHELSA data from {}".format(response))
#down = wget.download("https://os.zhdk.cloud.switch.ch/envicloud/chelsa/chelsa_V2/GLOBAL/annual/swb/CHELSA_swb_2013_V.2.1.tif")
download_data("../references/chelsa/test.txt", "../datasets/raw/chelsa/")

# Trial 1

In [12]:
def process_geotifs(input_dir, output_dir):
    """Iterates over geotif files, slices data for Europe and converts the data to equal area projection.

    :param input_dir: Path to folder with raw CHELSA data.
    :param: output_dir: Path for storing out files.

    Returns:
        None
    """
    # Define the EPSG codes for the input and output coordinate systems
    output_epsg = 3530  # ETRS89 / LAEA Europe equal area projection

    # Create the output directory if it doesn't already exist
    os.makedirs(output_dir, exist_ok=True)

    # Iterate over the geotif files in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith(".tif"):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)

            # Open the input geotif file
            dataset = gdal.Open(input_path)

            # Get the geotransform and projection information
            geotransform = dataset.GetGeoTransform()
            projection = dataset.GetProjection()

            # Convert the geotransform to a spatial reference object
            srs = osr.SpatialReference()
            srs.ImportFromWkt(projection)

            # Create a new spatial reference object for the output coordinate system
            output_srs = osr.SpatialReference()
            output_srs.ImportFromEPSG(output_epsg)

            # Create a transformation object to convert from the input to output coordinate system
            transform = osr.CoordinateTransformation(srs, output_srs)

            # Read the input geotif data
            data = dataset.ReadAsArray()

            # Compute the indices for the Europe region
            x_size = dataset.RasterXSize
            y_size = dataset.RasterYSize
            x_offset = int((-25.0 - geotransform[0]) / (geotransform[1]))
            y_offset = int((72.0 - geotransform[3]) / (geotransform[5]))
            x_count = (
                int(((-10.0 - geotransform[0]) / geotransform[1])) - x_offset
            )
            y_count = (
                int(((35.0 - geotransform[3]) / geotransform[5])) - y_offset
            )

            # Slice the data for the Europe region
            data = data[y_offset : y_offset + y_count, x_offset : x_offset + x_count]

            # Convert the data to the output coordinate system
            x_res = (geotransform[1] * x_count) / 1000.0
            y_res = (geotransform[5] * y_count) / 1000.0
            output_geotransform = (
                geotransform[0] + x_offset * geotransform[1],
                x_res,
                0.0,
                geotransform[3] + y_offset * geotransform[5],
                0.0,
                -y_res,
            )
            output_data = np.empty((y_count, x_count), dtype=np.float32)
            output_data[:] = np.nan
            output_dataset = gdal.GetDriverByName("GTiff").Create(
                output_path, x_count, y_count, 1, gdal.GDT_Float32
            )
            output_dataset.SetGeoTransform(output_geotransform)
            output_dataset.SetProjection(output_srs.ExportToWkt())
            gdal.ReprojectImage(
                dataset, output_dataset, projection, output_srs.ExportToWkt(), gdal
            )

process_geotifs("../datasets/raw/chelsa/", "../datasets/processed/chelsa/")

TypeError: in method 'ReprojectImage', argument 5 of type 'GDALResampleAlg'

# Trial 2

In [3]:
import glob
import numpy as np
import rasterio
import pyproj
import xarray as xr

def process_geotiffs_2(input_dir, output_file):
    # Define projection for Europe (Equal Area projection)
    europe_crs = pyproj.crs.CRS.from_string('+proj=laea +lat_0=52 +lon_0=10 +x_0=4321000 +y_0=3210000 +ellps=GRS80 +units=m +no_defs')
    
    # Define output grid shape
    xres, yres = 10000, 10000 # 10 km resolution
    xsize, ysize = 447, 547 # Grid shape for Europe
    
    # Create empty xarray dataset to store output
    data = np.zeros((1, ysize, xsize), dtype=np.float32)
    coords = {'latitude': np.linspace(85, 35, ysize), 'longitude': np.linspace(-25, 50, xsize)}
    ds = xr.Dataset({'data': (['time', 'latitude', 'longitude'], data)}, coords=coords)
    
    # Iterate over GeoTIFF files
    for tif_file in glob.glob(input_dir + '*.tif'):
        with rasterio.open(tif_file) as src:
            # Crop to Europe
            window = rasterio.windows.from_bounds(*src.bounds, europe_crs)
            data = src.read(1, window=window)
            transform = src.window_transform(window)
            
            # Reproject to Equal Area projection
            europe_transform = rasterio.warp.transform_bounds(src.crs, europe_crs, *src.bounds)
            europe_data, _ = rasterio.warp.reproject(data, transform, src.crs, europe_crs, shape=(ysize, xsize), 
                                                      bounds=europe_transform, resampling=rasterio.warp.Resampling.bilinear)
            
            # Downscale to 10 km grid
            europe_data = rasterio.warp.downscale_local_mean(europe_data, (int(yres/src.res[0]), int(xres/src.res[1])))
            
            # Add data to xarray dataset
            ds['data'][0, :, :] += europe_data.astype(np.float32)
    
    # Convert output to netCDF4 format
    ds.to_netcdf(output_file)

process_geotiffs_2("../datasets/raw/chelsa/", "../datasets/processed/chelsa/")

WindowError: A transform object is required to calculate the window

# Trial 3

In [2]:
!pip install rioxarray

  Using cached rioxarray-0.14.1-py3-none-any.whl (53 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.8/17.8 MB 11.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 10.2 MB/s eta 0:00:0000:0100:01
  Using cached affine-2.4.0-py3-none-any.whl (15 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
  Using cached snuggs-1.4.7-py3-none-any.whl (5.4 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)


In [ ]:
import os
import xarray as xr
import numpy as np
import rioxarray as rxr
import rasterio
from rasterio.enums import Resampling


def geotiff_to_netcdf_3(directory_path, output_file_path):
    
    # Define the bounding box for Europe
    europe_bounds = [-25, 35, 45, 75]

    # Iterate over all files in the directory
    for filename in os.listdir(directory_path):
        print("---------------------------------------------")
        output=filename.rsplit("_V")
        # Check if the file is a GeoTIFF
        if filename.endswith(".tif"):
            print(f"Reading {filename}...")

            # Open the GeoTIFF using rioxarray
            data = rxr.open_rasterio(os.path.join(directory_path, filename))
            print(f"Processing {filename} as xarray...")
           
            # Slice the data for Europe
            data_europe = data.rio.clip_box(*europe_bounds)
            print(f"Clipping {filename} for Europe...")

            # Reproject the data to ESPG-3530
            data_europe = data_europe.rio.reproject("EPSG:3035")
            print(f"Reprojecting {filename} as ESPG:3035...")
            
            # Resample data to 10x10 KM grids based on https://corteva.github.io/rioxarray/stable/examples/resampling.html
            downscale_factor = 10
            new_width = data_europe.rio.width * downscale_factor
            new_height = data_europe.rio.height * downscale_factor
            data_resampled= data_europe.rio.reproject(
                data_europe.rio.crs,
                shape=(new_height, new_width),
                resampling=Resampling.cubic,)
            print(f"Resamplimg {filename} using cubic algorithm...")

            # Convert to xarray DataArray to xrray Dataset
            dsout=data_resampled.to_dataset(name=f"{output[0]}")

            print(f"Final dataset for {output[0]}: ")
            print(dsout)

            # Convert the data to a netCDF4 file using xarray
            dsout.to_netcdf(
                output_file_path, 
                mode="a",
                group="/",
                format="NETCDF4"
            )
            print(f"{filename} processed and saved to {output_file_path} as {output[0]}")
            
geotiff_to_netcdf_3("../datasets/raw/chelsa/", "../datasets/processed/chelsa/resmpled_CHELSA_bio1-19.nc")

---------------------------------------------
Reading CHELSA_bio15_1981-2010_V.2.1.tif...
Processing CHELSA_bio15_1981-2010_V.2.1.tif as xarray...
Clipping CHELSA_bio15_1981-2010_V.2.1.tif for Europe...
Reprojecting CHELSA_bio15_1981-2010_V.2.1.tif as ESPG:3035...
Resamplimg CHELSA_bio15_1981-2010_V.2.1.tif using cubic algorithm...
Final dataset for CHELSA_bio15_1981-2010: 
<xarray.Dataset>
Dimensions:                 (x: 105610, y: 79730, band: 1)
Coordinates:
  * x                       (x) float64 1.215e+06 1.215e+06 ... 7.428e+06
  * y                       (y) float64 6.018e+06 6.018e+06 ... 1.328e+06
  * band                    (band) int64 1
    spatial_ref             int64 0
Data variables:
    CHELSA_bio15_1981-2010  (band, y, x) uint16 65535 65535 ... 65535 65535
CHELSA_bio15_1981-2010_V.2.1.tif processed and saved to ../datasets/processed/chelsa/resmpled_CHELSA_bio1-19.nc as CHELSA_bio15_1981-2010
---------------------------------------------
Reading CHELSA_bio14_1981-2010_

In [12]:
import xarray as xr
ds_disk = xr.open_dataset("../datasets/processed/chelsa/test.nc", engine='netcdf4')
ds_disk

KeyError: None

In [22]:
from netCDF4 import Dataset #pylint: disable=no-name-in-module
import numpy as np


f = Dataset("../datasets/processed/chelsa/CHELSA_bio1_1981-2010_.nc", "w", format="NETCDF4")
print(f)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): 
    variables(dimensions): 
    groups: 


In [1]:
import os
import xarray as xr
import numpy as np
import rioxarray as rxr
import rasterio


def geotiff_to_netcdf_4(directory_path, output_file_path):
    
    # Define the bounding box for Europe
    europe_bounds = [-25, 35, 45, 75]

    # Iterate over all files in the directory
    for filename in os.listdir(directory_path):
        print("---------------------------------------------")
        output=filename.rsplit("_V")
        # Check if the file is a GeoTIFF
        if filename.endswith(".tif"):
            print(f"Reading {filename}...")

            # Open the GeoTIFF using rioxarray
            data = rxr.open_rasterio(os.path.join(directory_path, filename))
            print(f"Processing {filename} as xarray...")
            dsout=data.to_dataset(name=f"{output[0]}")

            print(f"Final dataset for {output[0]}: ")
            print(dsout)

            # Convert the data to a netCDF4 file using xarray
            dsout.to_netcdf(
                output_file_path, 
                mode="a",
                group=f"{output[0]}",
                format="NETCDF4"
            )
            print(f"{filename} processed and saved to {output_file_path} as {output[0]}")
            
geotiff_to_netcdf_4("../datasets/raw/temp/", "../datasets/processed/chelsa/CHELSA_bio1-4.nc")    

---------------------------------------------
Reading CHELSA_bio2_1981-2010_V.2.1.tif...
Processing CHELSA_bio2_1981-2010_V.2.1.tif as xarray...
Final dataset for CHELSA_bio2_1981-2010: 
<xarray.Dataset>
Dimensions:                (band: 1, x: 43200, y: 20880)
Coordinates:
  * band                   (band) int64 1
  * x                      (x) float64 -180.0 -180.0 -180.0 ... 180.0 180.0
  * y                      (y) float64 84.0 83.99 83.98 ... -89.98 -89.99 -90.0
    spatial_ref            int64 0
Data variables:
    CHELSA_bio2_1981-2010  (band, y, x) uint16 ...
CHELSA_bio2_1981-2010_V.2.1.tif processed and saved to ../datasets/processed/chelsa/CHELSA_bio1-4.nc as CHELSA_bio2_1981-2010
---------------------------------------------
Reading CHELSA_bio3_1981-2010_V.2.1.tif...
Processing CHELSA_bio3_1981-2010_V.2.1.tif as xarray...
Final dataset for CHELSA_bio3_1981-2010: 
<xarray.Dataset>
Dimensions:                (band: 1, x: 43200, y: 20880)
Coordinates:
  * band                  